# Run coloc vs. liver and skin traits

In [14]:
setwd("/frazer01/projects/GTEx_v7/analysis/eqtls_deconvolution")

In [15]:
invisible(suppressWarnings(file.link("/frazer01/home//matteo/notebooks/eqtls_deconvolution_gtex//coloc_analysis.ipynb"     , "analysis/coloc_analysis.ipynb"    )))
invisible(suppressWarnings(file.link("/frazer01/home//matteo/notebooks/eqtls_deconvolution_gtex//cardiac_qtls_run_coloc.R" , "analysis/cardiac_qtls_run_coloc.R")))
invisible(suppressWarnings(file.link("/frazer01/home//matteo/notebooks/eqtls_deconvolution_gtex//cardiac_qtls_run_coloc.sh", "analysis/cardiac_qtls_run_coloc.sh")))

source("analysis/cardiac_qtls_packages.R"      )
source("analysis/cardiac_qtls_input_files.R"   )
source("analysis/cardiac_qtls_functions.R"     )
source("analysis/cardiac_qtls_input_data.R"    )
source("analysis/cardiac_qtls_load_metadata.R" )


Loading packages...
Loading input files...
Loading functions...
Loading input data...
Loading metadata...


In [16]:
dir.create(paste("private", "coloc",          sep = "/"), showWarnings = FALSE)

mains   = c("liver", "skin")
invisible(lapply(mains, function(main){dir.create(paste("private", "coloc", main, sep = "/"), showWarnings = FALSE)}))


In [17]:
tissues         = c("liver_original", "liver_cells", "liver2_cells", "liver3_cells", "skin_original", "skin_cells", "skin2_cells"  )
tissue2name     = data.frame(tissue = tissues, name = c("Liver (original)", "Liver (mouse cell populations)", "Liver (human cell populations)", "Liver (collapsed resolution)", "Skin (original)", "Skin (mouse cell populations)" , "Skin (collapsed resolution)" ), color = c("#FF7256", "#CD3700", "#800000", "#ff0000", "#87CEFA", "#000080", "#8080ff"), y = c(7,6,5,3,4,2,1), main = c(rep("liver", 4), rep("skin", 3)))
qtl_list        = lapply(tissues , function(x){read.table(paste("qtls/", x, "/analysis/egenes.", x, ".txt", sep = ""), header = TRUE)})
names(qtl_list) = tissues


In [18]:
get_egenes = function(qtl_list, tissue)
{
    x      = qtl_list[[tissue]]
    egenes = x[x$egene == TRUE, "gene_id"]
    
    return(data.frame(gene_id = egenes, tissue = tissue))
}
egenes = do.call("rbind", lapply(tissues, function(tissue){get_egenes(qtl_list, tissue)}))
egenes = merge(egenes, tissue2name)
egenes = unique(egenes[,c("gene_id", "main")])

write.table(egenes, paste("private", "coloc", "egenes.txt", sep = "/"), sep = "\t", quote = FALSE, row.names = FALSE, col.names = TRUE)


In [19]:
trait2test = list(liver = read_gwas_table("/home/mdonovan/gtex_deconvolution/tables/liver_gwas.txt"), skin = read_gwas_table("/home/mdonovan/gtex_deconvolution/tables/skin_gwas.txt"))



In [20]:
gene_ids  = sort(unique(egenes$gene_id))

In [22]:
run_qsub_coloc = function(qtl_folder, n_genes, run_qsub = FALSE, queue = "week", tc = 500)
{
    err_file = paste(qtl_folder, "log", "coloc.out", sep = "/")
    out_file = paste(qtl_folder, "log", "coloc.err", sep = "/")
    
    suppressWarnings(file.remove(err_file))
    suppressWarnings(file.remove(out_file))


    qsub_options = paste("-l", queue, 
                         "-pe", "smp 1", 
                         "-t", paste(1, n_genes, sep = "-"), 
                         "-tc", tc,
                         "-o", err_file,
                         "-e", out_file
                        )

    command = paste("qsub", qsub_options, paste(qtl_folder, "analysis/cardiac_qtls_run_coloc.sh", sep = "/"))

    if (run_qsub == TRUE ){message("Running qsub")}
    if (run_qsub == TRUE ){system(command)}
    if (run_qsub == FALSE){return(command)}
}

run_qsub_coloc(getwd(), length(gene_ids), run_qsub = FALSE, queue = "week", tc = 500)
run_qsub_coloc(getwd(), length(gene_ids), run_qsub = TRUE , queue = "week", tc = 500)


[1] "qsub -l week -pe smp 1 -t 1-16920 -tc 500 -o /frazer01/projects/GTEx_v7/analysis/eqtls_deconvolution/log/coloc.out -e /frazer01/projects/GTEx_v7/analysis/eqtls_deconvolution/log/coloc.err /frazer01/projects/GTEx_v7/analysis/eqtls_deconvolution/analysis/cardiac_qtls_run_coloc.sh"

Running qsub


In [12]:
coloc_by_main

function (main, taskid) 
{
    suppressMessages(source("analysis/cardiac_qtls_packages.R"))
    suppressMessages(source("analysis/cardiac_qtls_input_files.R"))
    suppressMessages(source("analysis/cardiac_qtls_functions.R"))
    suppressMessages(source("analysis/cardiac_qtls_input_data.R"))
    suppressMessages(source("analysis/cardiac_qtls_load_metadata.R"))
    gene2main = read.table(paste("private", "coloc", "egenes.txt", 
        sep = "/"), header = TRUE)
    gene_ids = sort(unique(gene2main$gene_id))
    gene_id = gene_ids[[taskid]]
    chrom = geneinfo[geneinfo$gene_id == gene_id, "chrom"]
    from = geneinfo[geneinfo$gene_id == gene_id, "start"] - qtl_distance
    to = geneinfo[geneinfo$gene_id == gene_id, "start"] + qtl_distance
    if (file.exists(paste("/frazer01/projects/GTEx_v7/analysis/eqtls_deconvolution/qtls/", 
        main, "_original/input/phenotypes/", gene_id, ".txt", 
        sep = "")) == TRUE) {
        traits = read_gwas_table(paste("/home/mdonovan/gtex_deconvolution/tables/", 
            main, "_gwas.txt", sep = ""))
        pv = get_ukbb_data(gene_id, main, chrom, from, to, traits, 
            "PV")
        trait2coloc = lapply(traits, function(trait) {
            trait2df(trait, pv)
        })
        qtl2coloc = lapply(tissue2name[tissue2name$main == main, 
            "tissue"], function(tissue) {
            qtl2df(gene_id, tissue)
        })
        names(trait2coloc) = traits
        names(qtl2coloc) = tissue2name[tissue2name$main == main, 
            "tissue"]
        to_coloc = c(qtl2coloc, trait2coloc)
        to_test = expand.grid(qtl = names(qtl2coloc), trait = names(trait2coloc), 
            stringsAsFactors = FALSE)
        run_coloc_all_interactions(gene_id, to_coloc, to_test, 
            main)
    }
}

# Test

In [182]:
mains   = c("liver", "skin")
taskid = 2
invisible(lapply(mains, function(main){dir.create(paste("private", "coloc", main, sep = "/"), showWarnings = FALSE)}))


In [183]:

mains   = c("liver", "skin")
taskid = 2

lapply(mains, function(main){coloc_by_main(main, taskid)})

PP.H0.abf PP.H1.abf PP.H2.abf PP.H3.abf PP.H4.abf 
  0.00128   0.01110   0.10200   0.88400   0.00215 
[1] "PP abf for shared variant: 0.215%"
PP.H0.abf PP.H1.abf PP.H2.abf PP.H3.abf PP.H4.abf 
 5.25e-05  1.23e-02  4.19e-03  9.83e-01  3.61e-04 
[1] "PP abf for shared variant: 0.0361%"
PP.H0.abf PP.H1.abf PP.H2.abf PP.H3.abf PP.H4.abf 
 5.34e-05  1.23e-02  4.26e-03  9.83e-01  3.64e-04 
[1] "PP abf for shared variant: 0.0364%"
PP.H0.abf PP.H1.abf PP.H2.abf PP.H3.abf PP.H4.abf 
   0.0626    0.5420    0.0363    0.3150    0.0444 
[1] "PP abf for shared variant: 4.44%"
PP.H0.abf PP.H1.abf PP.H2.abf PP.H3.abf PP.H4.abf 
  0.00256   0.60100   0.00149   0.34900   0.04550 
[1] "PP abf for shared variant: 4.55%"
PP.H0.abf PP.H1.abf PP.H2.abf PP.H3.abf PP.H4.abf 
  0.00260   0.60000   0.00151   0.34800   0.04700 
[1] "PP abf for shared variant: 4.7%"
PP.H0.abf PP.H1.abf PP.H2.abf PP.H3.abf PP.H4.abf 
   0.0717    0.6210    0.0305    0.2640    0.0124 
[1] "PP abf for shared variant: 1.24%"
PP.H0.abf

[[1]]
NULL

[[2]]
NULL

PP.H0.abf PP.H1.abf PP.H2.abf PP.H3.abf PP.H4.abf 
  0.00128   0.01110   0.10200   0.88400   0.00215 
[1] "PP abf for shared variant: 0.215%"
PP.H0.abf PP.H1.abf PP.H2.abf PP.H3.abf PP.H4.abf 
 5.25e-05  1.23e-02  4.19e-03  9.83e-01  3.61e-04 
[1] "PP abf for shared variant: 0.0361%"
PP.H0.abf PP.H1.abf PP.H2.abf PP.H3.abf PP.H4.abf 
 5.34e-05  1.23e-02  4.26e-03  9.83e-01  3.64e-04 
[1] "PP abf for shared variant: 0.0364%"
PP.H0.abf PP.H1.abf PP.H2.abf PP.H3.abf PP.H4.abf 
   0.0626    0.5420    0.0363    0.3150    0.0444 
[1] "PP abf for shared variant: 4.44%"
PP.H0.abf PP.H1.abf PP.H2.abf PP.H3.abf PP.H4.abf 
  0.00256   0.60100   0.00149   0.34900   0.04550 
[1] "PP abf for shared variant: 4.55%"
PP.H0.abf PP.H1.abf PP.H2.abf PP.H3.abf PP.H4.abf 
  0.00260   0.60000   0.00151   0.34800   0.04700 
[1] "PP abf for shared variant: 4.7%"
PP.H0.abf PP.H1.abf PP.H2.abf PP.H3.abf PP.H4.abf 
   0.0717    0.6210    0.0305    0.2640    0.0124 
[1] "PP abf for shared variant: 1.24%"
PP.H0.abf

# OLD

In [92]:
get_ukbb_data = function(gene_id, main, chrom, from, to, traits, type)
{
    outfile = paste("private", "moloc", main, paste(gene_id, type, "txt", sep = "."), sep = "/")
    command = paste(bcftools, "query", 
                    "-r", paste(gsub("chr", "", chrom), ":", from, "-", to, sep = ""),
                    "-s", paste(traits, collapse = ","),
                    "-f", paste('"%POS\t%ID\t%AF[\t%', type, ']\n"', sep = ""),
                    paste("/frazer01/publicdata/ukbiobank_20180801/ukbb2vcf/vcf", paste(chrom, "vcf", "gz", sep = "."), sep = "/"),
                    ">", 
                    outfile
                   )
    
    system(command)
    
    #return(command)
    
    indata           = fread(outfile, sep = "\t", header = FALSE, data.table = FALSE)
    colnames(indata) = c("pos", "id", "af", traits)
    
    #write.table(indata, outfile, quote = FALSE, sep = "\t", row.names = FALSE, col.names = TRUE)
    
    return(indata)
}

bt = get_ukbb_data(gene_id, main, chrom, from, to, traits, "BT")
se = get_ukbb_data(gene_id, main, chrom, from, to, traits, "SE")
pv = get_ukbb_data(gene_id, main, chrom, from, to, traits, "PV")


In [103]:
qtl2df = function(gene_id, tissue)
{
    infile = paste("qtls", tissue, "qtls", paste("all", gene_id, "txt", sep = "."), sep = "/")
    indata = fread(infile, sep = "\t", header = TRUE, data.table = FALSE)
    outdata = indata[,c("id", "beta", "se", "pval")]
    #colnames(outdata) = c("SNP", "BETA", "SE")
    outdata$z       = unlist(lapply(outdata$pval, function(p){zScores(p)}))
	outdata$varbeta = abs((outdata$beta / outdata$z)^2)
    outdata$z       = NULL
    
    return(outdata)
}

trait2df = function(trait, bt, se, pv)
{
    this         = data.frame(id = bt$id, af = bt$af, beta = bt[,trait], se = se[,trait], pval = pv[,trait])
    this$z       = unlist(lapply(this$pval, function(p){zScores(p)}))
	this$varbeta = abs((this$beta / this$z)^2)
    this$z       = NULL
    
    return(this)
}



In [104]:

trait2coloc        = lapply(traits, function(trait){trait2df(trait, bt, se, pv)})
qtl2coloc          = lapply(tissue2name[tissue2name$main == main, "tissue"], function(tissue){qtl2df(gene_id, tissue)})
names(trait2coloc) = traits
names(qtl2coloc)   = tissue2name[tissue2name$main == main, "tissue"]

to_coloc = c(qtl2coloc, trait2coloc)

In [112]:
id1 = "liver_cells"
id2 = names(trait2coloc)[[1]]

b1 = to_coloc[[id1]]
b2 = to_coloc[[id2]]

rownames(b1) = b1$id
rownames(b2) = b2$id

ids = intersect(b1$id, b2$id)

b1 = b1[ids,]
b2 = b2[ids,]

    
#    b1$maf = NULL
#    b2$maf = NULL
#    
#    n1 = nrow(b1)
#    n2 = nrow(b2)
#
#    b1 = merge(b1, to_coloc[[gene_id]][, c("id", "maf")])
#    b2 = merge(b2, to_coloc[[gene_id]][, c("id", "maf")])



    
coloc_mapped = coloc.abf(dataset1=list(snp = b1$id, pvalues = b1$pval, beta = b1$beta, varbeta = b1$varbeta, N = length(ids), MAF = b2$maf, sdY = 1, type = "quant"),
                         dataset2=list(snp = b2$id, pvalues = b2$pval, beta = b2$beta, varbeta = b2$varbeta, N = length(ids), MAF = b2$maf, sdY = 1, type = "quant"))   

coloc_mapped = coloc.abf(dataset1=list(snp = b1$id, pvalues = b1$pval, N = length(ids), type = "quant"),
                         dataset2=list(snp = b2$id, pvalues = b2$pval, N = length(ids), type = "quant"),
                         MAF     =b2$af
                        )   


PP.H0.abf PP.H1.abf PP.H2.abf PP.H3.abf PP.H4.abf 
       NA        NA        NA        NA        NA 
[1] "PP abf for shared variant: NA%"
PP.H0.abf PP.H1.abf PP.H2.abf PP.H3.abf PP.H4.abf 
 2.81e-52  7.72e-01  7.96e-53  2.19e-01  8.75e-03 
[1] "PP abf for shared variant: 0.875%"


In [109]:
str(b1)
str(b2)

'data.frame':	7303 obs. of  5 variables:
 $ id     : chr  "5_95211761_C_T" "5_95211865_A_T" "5_95211885_G_A" "5_95211992_C_T" ...
 $ beta   : num  -0.141 -0.206 -0.206 -0.206 -0.243 ...
 $ se     : num  0.221 0.256 0.256 0.256 0.656 ...
 $ pval   : num  0.867 0.236 0.236 0.236 0.803 ...
 $ varbeta: num  0.706 0.03 0.03 0.03 0.952 ...
'data.frame':	7303 obs. of  6 variables:
 $ id     : chr  "5_95211761_C_T" "5_95211865_A_T" "5_95211885_G_A" "5_95211992_C_T" ...
 $ af     : num  0.48327 0.77217 0.77424 0.77218 0.00849 ...
 $ beta   : num  2.26e-05 3.92e-05 3.46e-05 3.93e-05 1.09e-04 ...
 $ se     : num  4.63e-05 5.50e-05 5.53e-05 5.50e-05 2.54e-04 ...
 $ pval   : num  0.625 0.476 0.532 0.476 0.67 ...
 $ varbeta: num  2.14e-09 3.03e-09 3.06e-09 3.03e-09 6.48e-08 ...


In [69]:
test = moloc_test(tomoloc, prior_var = c(rep(0.5, length(qtl2moloc)), rep(0.1, length(trait2moloc))), priors = rep(1e-6, length(tomoloc)), save.SNP.info = FALSE)

ERROR: Error in moloc_test(tomoloc, prior_var = c(rep(0.5, length(qtl2moloc)), : Must give either MAF and N, or sdY


In [84]:
run_coloc = function(gene_id, to_coloc, id1, id2)
{
    b1 = to_coloc[[id1]]
    b2 = to_coloc[[id2]]
    
    b1$maf = NULL
    b2$maf = NULL
    
    n1 = nrow(b1)
    n2 = nrow(b2)
    
    b1 = merge(b1, to_coloc[[gene_id]][, c("id", "maf")])
    b2 = merge(b2, to_coloc[[gene_id]][, c("id", "maf")])
    
    
    coloc_mapped = coloc.abf(dataset1=list(snp = b1$id, pvalues = b1$pval, beta = b1$beta, varbeta = b1$varbeta, N = n1, MAF = b1$maf, sdY = 1, type = "quant"),
                             dataset2=list(snp = b2$id, pvalues = b2$pval, beta = b2$beta, varbeta = b2$varbeta, N = n2, MAF = b2$maf, sdY = 1, type = "quant"))   
    
    probs           = as.data.frame(t(coloc_mapped$summary))
    myres           = coloc_mapped$results
    myres           = coloc_mapped$results
    myres           = myres[, c(which(colnames(myres) == "snp"), ncol(myres))]
    colnames(myres) = c("snp_id", "pp_snp")
    myres           = myres[order(myres$pp_snp, decreasing = TRUE),]
    
    return(cbind(data.frame(id1 = id1, id2 = id2), probs, myres[1,]))
}




List of 3
 $ liver_original:'data.frame':	11768 obs. of  3 variables:
  ..$ id  : chr [1:11768] "5_95211761_C_T" "5_95211865_A_T" "5_95211885_G_A" "5_95211992_C_T" ...
  ..$ beta: num [1:11768] -0.0144 -0.2661 -0.2661 -0.2661 0.1761 ...
  ..$ se  : num [1:11768] 0.231 0.265 0.265 0.265 0.689 ...
 $ liver_cells   :'data.frame':	11768 obs. of  3 variables:
  ..$ id  : chr [1:11768] "5_95211761_C_T" "5_95211865_A_T" "5_95211885_G_A" "5_95211992_C_T" ...
  ..$ beta: num [1:11768] -0.141 -0.206 -0.206 -0.206 -0.243 ...
  ..$ se  : num [1:11768] 0.221 0.256 0.256 0.256 0.656 ...
 $ liver2_cells  :'data.frame':	11768 obs. of  3 variables:
  ..$ id  : chr [1:11768] "5_95211761_C_T" "5_95211865_A_T" "5_95211885_G_A" "5_95211992_C_T" ...
  ..$ beta: num [1:11768] -0.1321 -0.2122 -0.2122 -0.2122 -0.0464 ...
  ..$ se  : num [1:11768] 0.208 0.242 0.242 0.242 0.629 ...
